In [ ]:
import os
import json
import os
from tqdm import tqdm
import torch
from ultralytics import YOLO
import cv2

In [ ]:
# We will convert the BDD100K annotation JSON files into YOLO format. The output will be saved as .txt files in the specified directory.


# Class mapping (limited to vehicle categories used in this project)
CLASSES = {
    "car": 0,
    "bus": 1,
    "truck": 2,
}


"""
    Convert BDD100K annotation JSON file to YOLO format labels.

    Args:
        json_path (str): Path to BDD100K annotation JSON file
        image_dir (str): Directory containing corresponding images
        output_label_dir (str): Output directory for YOLO label files
"""


def convert(json_path, image_dir, output_label_dir):

    os.makedirs(output_label_dir, exist_ok=True)

    # Load annotation data
    with open(json_path) as f: 
        data = json.load(f)

    for item in tqdm(data):   # Iterate through all annotated images
        img_name = item["name"]
        img_path = os.path.join(image_dir, img_name)

        # Ignore unwanted categories
        if not os.path.exists(img_path): 
            continue

        label_path = os.path.join(
            output_label_dir,
            img_name.replace(".jpg", ".txt")
        )

        with open(label_path, "w") as out:
            for label in item["labels"]:
                if label["category"] not in CLASSES:  # Ignore unwanted categories
                    continue
                if "box2d" not in label: # Skip objects without bounding boxes
                    continue

                cls_id = CLASSES[label["category"]]
                box = label["box2d"]

                
                x1, y1, x2, y2 = box.values() # Convert bounding box to YOLO normalized format

                img_w, img_h = 1280, 720   # BDD100K standard image resolution

                xc = ((x1 + x2) / 2) / img_w
                yc = ((y1 + y2) / 2) / img_h
                w = (x2 - x1) / img_w
                h = (y2 - y1) / img_h

                out.write(f"{cls_id} {xc} {yc} {w} {h}\n")

# ---- train ----
convert(
    json_path="bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json",
    image_dir="bdd100k/bdd100k/images/100k/train",
    output_label_dir="yolo/labels/train"
)

# ---- val ----
convert(
    json_path="bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_val.json",
    image_dir="bdd100k/bdd100k/images/100k/val",
    output_label_dir="yolo/labels/val"
)


100%|██████████| 10000/10000 [00:02<00:00, 3878.73it/s]


In [ ]:
# --------------------------------------------------
# Check PyTorch & CUDA environment
# --------------------------------------------------


print("torch:", torch.__version__)
print("cuda:", torch.cuda.is_available())
print("gpu:", torch.cuda.get_device_name(0))


torch: 2.3.1+cu121
cuda: True
gpu: NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [ ]:
# --------------------------------------------------
# Initial YOLO fine-tuning (short training)
# --------------------------------------------------


model = YOLO("yolo11s_2.pt")

results = model.train(
    data="data.yaml",
    epochs=100,
    patience=5,      
    weight_decay=5e-4,
    dropout=0.1, 
    project="car_finetune",
    name="bdd100k_v1" ,
    workers=4,
    device=0,
    freeze=10,
    imgsz=640,
    batch=8,
    lr0=0.001,
)


Ultralytics 8.3.237  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s_2.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bdd100k_v13, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=T

KeyboardInterrupt: 

In [ ]:

model = YOLO('E:\programming\Github-projects\carYoloforGitgub\car_finetune\\bdd100k_v13\weights\\best.pt')
results = model.predict(source='E:\programming\Github-projects\carYoloforGitgub\dataset\images\\New folder\cc0e87d8-915720ff.jpg', conf=0.5, iou=0.5, save=True, save_txt=True)
print(results)



image 1/1 E:\programming\Github-projects\carYoloforGitgub\dataset\images\New folder\cc0e87d8-915720ff.jpg: 384x640 7 cars, 89.1ms
Speed: 5.3ms preprocess, 89.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict4
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict4\labels
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'car', 1: 'bus', 2: 'truck'}
obb: None
orig_img: array([[[ 3,  2,  4],
        [ 3,  2,  4],
        [ 3,  2,  4],
        ...,
        [24, 19, 20],
        [24, 19, 20],
        [24, 19, 20]],

       [[ 3,  2,  4],
        [ 3,  2,  4],
        [ 3,  2,  4],
        ...,
        [24, 19, 20],
        [24, 19, 20],
        [24, 19, 20]],

       [[ 3,  2,  4],
        [ 3,  2,  4],
        [ 3,  2,  4],
        ...,
        [24, 19, 20],
  

In [ ]:

model = YOLO('E:\programming\Github-projects\carYoloforGitgub\car_finetune\\bdd100k_v13\weights\\best.pt')  

results = model.predict(source='E:\programming\Github-projects\carYoloforGitgub\\video.mp4', conf=0.5, iou=0.5, save=True, save_txt=True)
print(results)




WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/750) E:\programming\Github-projects\carYoloforGitgub\video.mp4: 384x640 2 cars, 10.7ms
video 1/1 (frame 2/750) E:\programming\Github-projects\carYoloforGitgub\video.mp4: 384x640 2 cars, 9.6ms
video 1/1 (frame 3/750) E:\programming\Github-projects\carYoloforGitgub\video.mp4: 384x640 2 cars, 15.1ms
video 1/1 (frame 4/750) E:\programming\Github-projects\carYoloforGitgub\video.mp4: 384x640 2 cars, 10.4ms
video 1/1 (frame 5/750) E:\programmi

In [ ]:
# --------------------------------------------------
# Vehicle counting using tracking (real-time)
# --------------------------------------------------

cap = cv2.VideoCapture(video_path)
 
FONT = cv2.FONT_HERSHEY_DUPLEX
def draw_text(text, x, y):
       
    cv2.putText(frame, text, (x+2, y+2), FONT, 0.9, (0,0,0), 3)
       
    cv2.putText(frame, text, (x, y), FONT, 0.9, (255,255,255), 2)

if not cap.isOpened():
    print("Error: Could not open video file.")
else:
    print("Video file opened successfully!")


LINE_Y = 300  # Horizontal counting line

# Counters
car_count = 0
bus_count = 0
truck_count = 0
counted_ids = set()

while True:

    ret, frame = cap.read()
    
    if not ret:
        cv2.destroyAllWindows()
        break

    # Run detection + tracking
    results = model.track(
        frame,
        persist=True,
        tracker="bytetrack.yaml",
        classes=[0,1,2],      # فقط car
        verbose=False
    )[0]
    

     # Draw counting line
    cv2.line(
        frame,
        (0, LINE_Y),
        (frame.shape[1], LINE_Y),
        (0, 0, 255),
        1
    )

    # Count vehicles crossing the line
    if results.boxes.id is not None:
        for box, cls, tid in zip(
            results.boxes.xyxy,
            results.boxes.cls,
            results.boxes.id
        ):
            y1, y2 = int(box[1]), int(box[3])
            cy = (y1 + y2) // 2
            tid = int(tid)
            cls = int(cls)

            if cy > LINE_Y and tid not in counted_ids:
                counted_ids.add(tid)

                if cls == 0:
                    car_count += 1
                elif cls == 2:
                    bus_count += 1
                elif cls == 3:
                    truck_count += 1

    # Display counters
    draw_text(f"Cars   : {car_count}",   20, 60)
    draw_text(f"Buses  : {bus_count}",  20, 95)
    draw_text(f"Trucks : {truck_count}",20,130)


    cv2.imshow("video",frame)
    
    if cv2.waitKey(1) == ord("q"):
        cv2.destroyAllWindows()
        break


Video file opened successfully!
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6\labels
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6\labels
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6\labels
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6\labels
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6
1 label saved to E:\programming\Github-projects\carYoloforGitgub\runs\detect\predict6\labels
Results saved to E:\programming\Github-projects\carYoloforGitgub\runs\det